In [ ]:
# Import all necessary packages #

import pennylane as qml
from pennylane.qnodes import PassthruQNode

import scipy as sp

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import numpy as np

import qutip as qt
from qutip import Bloch, basis

In [ ]:
# Import python modules from vqe_function_states #

from vqe_function_states import init_cost_fn, run_vqe

In [ ]:
dev_1 = qml.device("default.qubit", wires = 1)

def circuit(params, wires=0):
    qml.RX(params[0], wires=wires)
    qml.RY(params[1], wires=wires)

In [ ]:
# Instantiate the cost function #

cost_function = init_cost_fn(dev = dev_1,
                             circuit = circuit, 
                             coeffs = [1,1],
                             obs = [qml.PauliX(0), qml.PauliZ(0)])

In [ ]:
init_params = np.array([3.97507603, 3.00854038])    

In [ ]:
# Run VQE iteration for quantum natural gradient descent #

qngd_cost_history, qngd_param_history, qngd_state_history, qngd_steps = run_vqe(cost_fn=cost_function,
                                                                                max_iter=500,
                                                                                initial_params=init_params,
                                                                                opt='QNGOptimizer',
                                                                                opt_step=0.01,
                                                                                dev = dev_1,
                                                                                diag_approx=True)

In [ ]:
# Run VQE iteration for gradient descent #

gd_cost_history, gd_param_history, gd_state_history, gd_steps = run_vqe(cost_fn=cost_function,
                                                                        max_iter=500,
                                                                        initial_params=init_params,
                                                                        opt='GradientDescentOptimizer',
                                                                        opt_step=0.01,
                                                                        dev = dev_1,
                                                                        diag_approx=False)

In [ ]:
# Track energy history for both optimization methods #

plt.style.use("seaborn")
plt.plot(gd_cost_history, "b", label="Gradient descent")
plt.plot(qngd_cost_history, "g", label="Quantum natural gradient descent")

plt.ylabel("Cost function value")
plt.xlabel("Optimization steps")
plt.legend()
plt.show()

In [ ]:
# Import python modules from visualization_function #

from visualization_function import plot_params, prepare_plot_states, plot_bloch_sphere

In [ ]:
# Visualize the optimization path in the parameter space using a contour plot #

plot_params(gd_param_history=gd_param_history,
            qngd_param_history=qngd_param_history,
            plot_interval=10,
            figsize=[6,6],
            linewidth=1)

In [ ]:
# Convert statevectors into coordinates for plotting on the bloch sphere #

qngd_x, qngd_y, qngd_z = prepare_plot_states(state_history=qngd_state_history)
gd_x, gd_y, gd_z = prepare_plot_states(state_history=gd_state_history)

In [ ]:
# Plot quantum natural gradient descent optimization path on the bloch sphere #

plot_bloch_sphere(opt_coords_x=qngd_x, 
                  opt_coords_y=qngd_y, 
                  opt_coords_z=qngd_z, 
                  plot_interval = 10, 
                  plot_color = 'g'
                  figsize = [8,8], 
                  pointsize = 20)

In [ ]:
# Plot vanilla gradient descent optimization path on the bloch sphere #

plot_bloch_sphere(opt_coords_x=gd_x, 
                  opt_coords_y=gd_y, 
                  opt_coords_z=gd_z, 
                  plot_interval = 10, 
                  plot_interval= 'k'
                  figsize = [8,8], 
                  pointsize = 20)